In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam


In [2]:
csv_files = ['ADANIPORTS.csv', 'ASIANPAINT.csv','AXISBANK']
models = []


In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [7]:

# Define a function to fetch stock data from Yahoo Finance
def fetch_stock_data(stock_symbol, start_date, end_date):
    df = yf.download(stock_symbol, start=start_date, end=end_date)
    return df

# Define a function to create sequences from data
def create_sequences(data, sequence_length):
    sequences, labels = [], []
    for i in range(len(data) - sequence_length):
        seq = data[i:i+sequence_length]
        label = data[i+sequence_length]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

# Define a function to create and train an LSTM model for a given stock
def create_train_lstm_model(stock_data, sequence_length, epochs=100, batch_size=32):
    # Normalize the data
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(stock_data[['Close']].values)

    # Split the data into training and testing sets
    train_size = int(len(scaled_data) * 0.8)
    train_data, test_data = scaled_data[:train_size], scaled_data[train_size:]

    # Prepare the data for LSTM training
    X_train, y_train = create_sequences(train_data, sequence_length)
    X_test, y_test = create_sequences(test_data, sequence_length)

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(sequence_length, 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

    # Evaluate the model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    return model, scaler, mse

# Define the stocks you want to analyze
stocks = [
    'AAPL', 'MSFT', 'AMZN', 'GOOGL', 'TSLA', 'BRK.B', 'JPM', 'V', 'JNJ',
    'WMT', 'PG', 'VZ', 'PFE', 'NVDA', 'MA', 'DIS', 'UNH', 'HD', 'BAC',
    'MRK', 'XOM', 'CSCO', 'PYPL', 'T', 'CRM', 'KO', 'NKE', 'ORCL', 'ADBE',
    'INTC', 'CMCSA', 'VFC', 'SBUX', 'CVX', 'MCD', 'TMUS', 'ACN', 'MMM', 'IBM',
    'LMT', 'NOW', 'TMO', 'PEP', 'UNP', 'CAT', 'COST', 'LLY', 'ABBV', 'AMD'
]

# Define date range for historical data
start_date = '2022-06-01'
end_date = '2023-01-01'

# Define sequence length
sequence_length = 9

# Create and train LSTM models for each stock
models = {}
for stock_symbol in stocks:
    stock_data = fetch_stock_data(stock_symbol, start_date, end_date)
    model, scaler, mse = create_train_lstm_model(stock_data, sequence_length)
    models[stock_symbol] = {'model': model, 'scaler': scaler, 'mse': mse}

# Make predictions using individual models for each stock
for stock_symbol in stocks:
    stock_data = fetch_stock_data(stock_symbol, start_date, end_date)
    scaled_data = models[stock_symbol]['scaler'].transform(stock_data[['Close']].values)
    X, y = create_sequences(scaled_data, sequence_length)
    predictions = models[stock_symbol]['model'].predict(X)

    # Inverse transform the predictions to get the actual stock prices
    predicted_prices = models[stock_symbol]['scaler'].inverse_transform(predictions)

    # Print or use the predicted prices as needed
    print(f"Predictions for {stock_symbol}:")
    print(predicted_prices)



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Predictions for AAPL:
[[141.49606]
 [138.63937]
 [137.1113 ]
 [135.0363 ]
 [134.19151]
 [134.95525]
 [135.9598 ]
 [137.45842]
 [139.2218 ]
 [140.68375]
 [141.28542]
 [141.96968]
 [141.52428]
 [141.13805]
 [141.2929 ]
 [141.84174]
 [143.43997]
 [145.27167]
 [146.51959]
 [147.37393]
 [147.62459]
 [148.03139]
 [